In [16]:
import dlib

import torch
import torch.nn as nn
import torch.nn.functional as F

from skimage import io

In [20]:
path_to_file = 'data/Menpo_68p/test/aflw__face_39844.jpg'

In [23]:
def face_bbox(file_path: str) -> list[float]:
    detector = dlib.get_frontal_face_detector()
    img = io.imread(file_path)
    dets, _, _ = detector.run(img, 1, -1)
    try:
        face_bbox = [dets[0].left(), dets[0].top(), dets[0].right(), dets[0].bottom()]
    except IndexError:
        print(f"Following face wasn't recognized {file_path}")
    return face_bbox

In [ ]:
def get_model(min_size=800):
    # initialize the model
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True,
                                                                   num_keypoints=17, 
                                                                   min_size=min_size)

    return model

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform(m.weight.data)
        nn.init.constant(m.bias, 0.1)

In [ ]:
class ONet(nn.Module):
    ''' RNet '''

    def __init__(self,is_train=False, use_cuda=True):
        super(ONet, self).__init__()
        self.is_train = is_train
        self.use_cuda = use_cuda
        # backend
        self.pre_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1),  # conv1
            nn.PReLU(),  # prelu1
            nn.MaxPool2d(kernel_size=3, stride=2),  # pool1
            nn.Conv2d(32, 64, kernel_size=3, stride=1),  # conv2
            nn.PReLU(),  # prelu2
            nn.MaxPool2d(kernel_size=3, stride=2),  # pool2
            nn.Conv2d(64, 64, kernel_size=3, stride=1),  # conv3
            nn.PReLU(), # prelu3
            nn.MaxPool2d(kernel_size=2,stride=2), # pool3
            nn.Conv2d(64,128,kernel_size=2,stride=1), # conv4
            nn.PReLU() # prelu4
        )
        self.conv5 = nn.Linear(128*2*2, 256)  # conv5
        self.prelu5 = nn.PReLU()  # prelu5
        # lanbmark localization
        self.conv6_3 = nn.Linear(256, 136)
        # weight initiation weih xavier
        self.apply(weights_init)

    def forward(self, x):
        # backend
        x = self.pre_layer(x)
        x = x.view(x.size(0), -1)
        x = self.conv5(x)
        x = self.prelu5(x)
        # detection
        landmark = self.conv6_3(x)
        if self.is_train is True:
            return landmark